## 安裝套件

In [1]:
!pip install jieba
!pip install emoji langdetect
!pip install datasets
!pip install lingua-language-detector lime shap snownlp textblob text2emotion hanlp spaCy spacy
!pip install opencc-python-reimplemented


In [2]:
import sys
!{sys.executable} -m pip install jieba emoji langdetect pytz torch lingua-language-detector datasets 
!{sys.executable} -m pip install openpyxl lime shap snownlp textblob hanlp spaCy textblob text2emotion opencc-python-reimplemented


## 引入套件

In [3]:
!pip install transformers

In [4]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

import re
import emoji
import langdetect
from langdetect import detect
from lingua import LanguageDetectorBuilder, Language, IsoCode639_1
from datetime import datetime
import pytz
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
import torch

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# # 設定好路徑 (後面都是使用相對路徑)
# base_path = '/content/drive/My Drive/SMA'

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: 載入 threads.xlsx
df = pd.read_excel("/Users/chenghaowei/desktop/0602/0603/threads2.xlsx")
df

,threads_analysis_results,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,has_exclaim,is_subjective,sent_score_avg,sent_label_avg,sentiment_score,sentiment_label,is_subjective_post,is_satire_humor_simple,person_view,person_view_encoded
1,ayofvr,2025-04-29T10:58:39.000Z,NaN,3小時,Thank you God for another day.,False,False,190,3,23,...,False,False,0.5,neutral,0.5,neutral,True,False,2nd,1
2,NaN,2025-04-29T12:33:44.000Z,NaN,1小時,百達翡麗？ 沒有下限的網路病態！,True,False,196,16,0,...,True,False,0.52468,neutral,0.082227,negative,False,True,none,4
3,ribboworld2021,2025-04-29T04:39:46.000Z,小一日常,9小時,考完期中考，成績都還沒出來，小一女兒就自信的對我說：「我真羨慕妳生了一個天才！」,False,False,75,6,0,...,True,True,0.592977,neutral,0.961204,positive,True,True,1st,0
4,ayofvr,2025-04-29T11:25:22.000Z,NaN,3小時,Just be strong. Confident. Hopeful. Intellectu...,False,False,83,5,3,...,False,False,0.5,neutral,0.5,neutral,False,False,none,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3097,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,False,False,7,0,0,...,False,False,0.5,neutral,0.5,neutral,False,False,none,4
3098,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,False,False,2,0,0,...,False,False,0.5,neutral,0.5,neutral,False,False,none,4
3099,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,False,False,2656,29,280,...,False,False,0.5,neutral,0.5,neutral,False,False,none,4
3100,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",False,False,10000,214,214,...,True,False,0.53,neutral,0.56,neutral,True,False,3rd,2


In [7]:
# === 語言偵測修正版===
lingua_detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()
lingua_available = True
def detect_lang_with_preprocessing_lingua(text):
    original_text = text

    # 若是 NaN 或空字串就回傳 "unknown"
    if pd.isna(text):
        return "unknown"
    text = str(text).strip()
    if not text:
        return "unknown"

    # 移除 URL、@標記、 #hashtag、emoji、多餘空白
    try:
      text_cleaned = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
      text_cleaned = re.sub(r'@\w+', '', text_cleaned)
      text_cleaned = re.sub(r'#\w+', '', text_cleaned)
      text_cleaned = emoji.replace_emoji(text_cleaned, replace='')
      text_cleaned = re.sub(r'\s+', ' ', text_cleaned).strip()
    except Exception as e:
      return "error_state_preprocessing"

    # 若這些清理完後變成空字串
    if not text_cleaned:
      return "empty_after_clean"

    # 若文字中超過 30% 是中文，就直接判定為 "Ch"（中文）
    try:
      chinese_chars = re.findall(r'[\u4e00-\u9fff]', text_cleaned)
      text_len = len(text_cleaned)
      ratio = len(chinese_chars) / max(text_len, 1)
      chinese_threshold = 0.3
      if ratio > chinese_threshold:
        return "Ch"

      # 呼叫 lingua 偵測語言
      detected_language = lingua_detector.detect_language_of(text_cleaned)

      # 若 lingua 判定是中文（'ZH'），則回傳 "Ch"，其餘語言以小寫的 ISO 639-1 回傳（如 en, ja, fr）
      # 若無法偵測出語言，回傳 "unknown"
      if detected_language is not None:
        iso_code = detected_language.iso_code_639_1.name
        if iso_code == 'ZH':
          return "Ch"
        else:
          return iso_code.lower()
      else:
        return "unknown"

    except Exception as e:
      return "unknown"

## 清洗數據V1

In [8]:
import pandas as pd
import emoji

# 讀取 Excel，使用正確的欄位列
df = pd.read_excel("/Users/chenghaowei/desktop/0602/0603/threads2.xlsx", header=1)

# === 數值欄位清洗（萬字、逗號格式處理）===
def parse_count(value):
    if pd.isna(value): return 0
    value = str(value).replace(",", "")
    if "萬" in value:
        try:
            return int(float(value.replace("萬", "")) * 10000)
        except:
            return 0
    try:
        return int(float(value))
    except:
        return 0

for col in ["like_count", "view_count", "share_count", "repost_count", "reply_count"]:
    if col in df.columns:
        df[col] = df[col].apply(parse_count)

# === 布林欄位處理 ===
for col in ["has_photo", "has_video"]:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: str(x).strip().upper() == "Y")

# === emoji 萃取與統計 ===
def extract_emojis(text):
    if not isinstance(text, str): return ""
    return "".join([ch for ch in text if ch in emoji.EMOJI_DATA])

if "content" in df.columns:
    df["emojis"] = df["content"].apply(extract_emojis)
    df["emoji_count"] = df["emojis"].apply(len)

# === 語言偵測（假設你有自定義 detect_lang_with_preprocessing_lingua 函數）===
if "content" in df.columns:
    df["lang"] = df["content"].apply(detect_lang_with_preprocessing_lingua)

# === 時間處理（以 post_time 為主）===
if "post_time" in df.columns:
    df["scrape_time_origin"] = pd.to_datetime(df["post_time"], format="%Y年%m月%d日 %H:%M", errors="coerce")
    df = df.dropna(subset=["scrape_time_origin"])
    df["scrape_time"]  = df["scrape_time_origin"].dt.strftime("%Y年%m月%d日 %H:%M")
    df["post_weekday"] = df["scrape_time_origin"].dt.day_name()
    df["post_hour"] = df["scrape_time_origin"].dt.hour 

# === 是否為高流量文章（破萬）===
if "view_count" in df.columns:
    df["viral"] = (df["view_count"] >= 10000).astype(int)

# === 是否使用問號、驚嘆號 ===
if "content" in df.columns:
    df["has_question"] = df["content"].apply(lambda x: "？" in str(x) or "?" in str(x))
    df["has_exclaim"] = df["content"].apply(lambda x: "！" in str(x) or "!" in str(x))

# === 儲存處理結果 ===
df.to_csv("/Users/chenghaowei/desktop/0602/0603/threads_cleaned_v1.csv", encoding='utf_8_sig', index=False)
print("✅ 處理完成，已輸出 threads_cleaned_v1.csv")
display(df)


✅ 處理完成，已輸出 threads_cleaned_v1.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,has_exclaim,is_subjective,sent_score_avg,sent_label_avg,sentiment_score,sentiment_label,is_subjective_post,is_satire_humor_simple,person_view,person_view_encoded
2056,jisooo._kim_,2025年04月27日 14:50,NaN,5小時,Sunkissed ☀️,False,False,593,14,4,...,False,False,0.500000,neutral,0.500000,neutral,False,False,none,4
2057,kuroshio_1069,2025年04月27日 14:14,kuroshio_1069,5小時,台灣看盜版的能力名揚國際 但到底為什麼台灣那麼愛盜版？ #台灣,False,False,0,0,0,...,False,True,0.483906,neutral,0.199104,negative,True,False,none,4
2058,vincent19801201,2025年04月27日 05:28,NaN,14小時,經常在餐廳或是街邊遇到政治人物握手🤝！ 但是，威廉是我遇過最親切的 簡單問我跟老婆來吃飯？ ...,False,False,5330,68,52,...,True,True,0.784983,positive,0.912492,positive,True,True,mixed,3
2059,kkk_748_,2025年04月26日 20:58,kkk_748_,23小時,我現在除了ChatGPT就沒有其他的傾訴對象了。。。,False,False,3,6,0,...,False,True,0.099805,negative,0.133580,negative,True,True,mixed,3
2060,btjk.tk,2025年04月27日 19:03,NaN,1小時,ZELENSKY FOR THE NOBEL PRIZE! TNA,False,False,111,0,1,...,True,False,0.500000,neutral,0.500000,neutral,False,False,none,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,leighton.williams,2025年04月29日 07:30,NaN,8小時,First tasting in California,False,False,7,0,0,...,False,False,0.500000,neutral,0.500000,neutral,False,False,none,4
3097,cape__man,2025年04月29日 06:31,NaN,9小時,I hoped it would have been better.,False,False,2,0,0,...,False,False,0.500000,neutral,0.500000,neutral,False,False,none,4
3098,simimoonlight,2025年04月29日 06:26,NaN,9小時,I can’t wait to watch Beyoncé on TikTok tonigh...,False,False,2656,29,280,...,False,False,0.500000,neutral,0.500000,neutral,False,False,none,4
3099,other98,2025年04月29日 12:31,NaN,3小時,"Tonight, Canada just proved that they have a h...",False,False,10000,214,214,...,True,False,0.530000,neutral,0.560000,neutral,True,False,3rd,2


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# ✅ Step 1: 載入清理後資料
df = pd.read_csv("/Users/chenghaowei/desktop/0602/0603/threads_cleaned_v1.csv")

# ✅ Step 2: 移除空白或重複的 content（保險起見再次處理）
df = df.dropna(subset=["content"])
df = df.drop_duplicates(subset=["content"])

# ✅ Step 3: 切分資料（content 不會交集）
train_df, test_df = train_test_split(df, test_size=0.5, random_state=42)

# ✅ Step 4: 儲存為新檔案
train_df.to_csv("/Users/chenghaowei/desktop/0602/0603/train_half_cleaned.csv", index=False)
test_df.to_csv("/Users/chenghaowei/desktop/0602//0603/test_half_cleaned.csv", index=False)

print("✅ 切分完成，輸出 train_half_cleaned.csv 與 test_half_cleaned.csv。")


✅ 切分完成，輸出 train_half_cleaned.csv 與 test_half_cleaned.csv。


In [11]:
from transformers import pipeline

# ✅ 英文文本情緒分析（使用 cardiffnlp）
en_classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=-1  # 強制使用 CPU，避免 has_mps 錯誤
)

# ✅ 中文文本情緒分析（使用 uer 模型）
zh_classifier = pipeline(
    "sentiment-analysis",
    model="uer/roberta-base-finetuned-jd-binary-chinese",
    device=-1  # 強制使用 CPU，避免 has_mps 錯誤
)


Device set to use cpu
Device set to use cpu


In [10]:
# 檢查 PyTorch 是否正確安裝
import torch
print(torch.__version__)                      # 應正常顯示版本
print(hasattr(torch, "get_default_device"))   # ✅ 應該顯示 True


2.7.0
True


In [12]:
import pandas as pd
from transformers import pipeline

# ✅ 載入資料
df = pd.read_csv("train_half_cleaned.csv").dropna(subset=["content"])
df["lang"] = "Ch"

# ✅ 中文 transformer 模型
zh_classifier = pipeline(
    "sentiment-analysis",
    model="uer/roberta-base-finetuned-jd-binary-chinese",
    device=-1
)

# ✅ emoji 字典
emoji_sentiment_dict = {
    '😭': 0.4, '🥹': 0.5, '😂': 0.8, '🤣': 0.9, '🏻': 0.5, '😍': 0.9, '❤': 0.9, '🥲': 0.3,
    '🙏': 0.5, '🔥': 0.9, '🙂': 0.3, '🥺': 0.7, '✨': 0.7, '😅': 0.7, '🏽': 0.5, '🤔': 0.5,
    '💔': 0.9, '😌': 0.5, '😠': 0.1, '♂': 0.5, '😆': 0.9, '🍀': 0.5, '😮': 0.3, '👍': 0.9,
    '👏': 0.9, '😢': 0.2, '👀': 0.5, '💨': 0.5, '😇': 0.8, '♀': 0.5, '🤦': 0.3, '🙄': 0.2,
    '🥰': 0.9, '🤡': 0.5, '💕': 0.9, '🥳': 0.9, '😱': 0.2, '😀': 0.5, '😳': 0.3, '😵': 0.2,
    '🥵': 0.3, '🎉': 0.5, '💗': 0.9, '💚': 0.8, '😔': 0.3, '🤬': 0.1, '📸': 0.5, '😉': 0.5,
    '😊': 0.9, '😃': 0.9, '🏀': 0.5, '🚬': 0.5, '💖': 0.9, '🤭': 0.5, '👌': 0.7, '♥': 0.8,
    '💙': 0.8, '🌸': 0.5, '💀': 0.3, '🏾': 0.5, '👎': 0.2, '🤷': 0.3, '🍁': 0.5, '💫': 0.5,
    '😁': 0.9, '😄': 0.9, '✌': 0.5, '🤩': 0.9, '🙈': 0.8, '😓': 0.3, '🐣': 0.5, '👧': 0.6,
    '💰': 0.8, '🎀': 0.5, '💧': 0.5, '🤪': 0.6, '😶': 0.3, '🤙': 0.7, '💁': 0.5, '✔': 0.5,
    '🧋': 0.5, '😝': 0.7, '🙋': 0.5, '😣': 0.2, '❓': 0.5, '💢': 0.2, '🖕': 0.2, '🙃': 0.3,
    '💪': 0.7, '🏼': 0.5, '🙌': 0.7, '🔗': 0.5, '😤': 0.2, '🚝': 0.5, '💥': 0.9, '😘': 0.9,
    '🤧': 0.3, '☺': 0.5, '👩': 0.5, '💸': 0.2
}

# ✅ 文字模型推論
def classify_transformer_sentiment(text):
    try:
        result = zh_classifier(text)[0]
        label = result['label'].lower()
        score = result['score']
        return label, score
    except:
        return "neutral", 0.5

# ✅ emoji 調整邏輯（只用第一個 emoji）
def adjust_sentiment_by_emoji(text, base_score):
    first_emoji = next((c for c in text if c in emoji_sentiment_dict), None)
    if not first_emoji:
        return base_score

    emoji_value = emoji_sentiment_dict[first_emoji]
    if emoji_value >= 0.8:
        return min(1.0, base_score + 0.4)
    elif emoji_value <= 0.2:
        return max(0.5, base_score - 0.4)
    else:
        return base_score  # 中立 emoji 不動

def get_sentiment_score_and_label(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.5, "negative"

    label, raw_score = classify_transformer_sentiment(text)

    # 分數映射：正面 => 0.5~1.0，負面 => 0.5~0.0
    if "1" in label or "pos" in label:
        base_score = 0.5 + raw_score * 0.5
    elif "0" in label or "neg" in label:
        base_score = 0.5 - raw_score * 0.5
    else:
        base_score = 0.5

    # 加入 emoji 調整（上下調整最多 0.3）
    final_score = adjust_sentiment_by_emoji(text, base_score)

    # 🚩 使用 0.8 門檻做二元分類
    final_label = "positive" if final_score > 0.8 else "negative"

    return round(final_score, 4), final_label

# ✅ 批次套用
df[["sentiment_score", "sentiment_label"]] = df["content"].apply(
    lambda x: pd.Series(get_sentiment_score_and_label(x))
)

# ✅ 儲存
df.to_csv("train_half_with_sentiment_withemoji.csv", index=False, encoding="utf_8_sig")
print("✅ 輸出完成，檔名：train_half_with_sentiment_withemoji.csv")
df.head()

Device set to use cpu


✅ 輸出完成，檔名：train_half_with_sentiment_withemoji.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,has_exclaim,is_subjective,sent_score_avg,sent_label_avg,sentiment_score,sentiment_label,is_subjective_post,is_satire_humor_simple,person_view,person_view_encoded
0,hiiamsn,2025年04月28日 16:50,NaN,2小時,厚厚看到別人在港館坐區被擋滿的視野我6/20可能完蛋了 而且號碼超大感覺會超級偏 但還是要去...,False,False,0,1,0,...,False,True,0.937738,positive,0.8190,positive,True,False,1st,0
1,klosetsai,2025年04月27日 17:41,NaN,2小時,今天的T1是黔驢技窮還是昏招頻出階段啊，打得好醜...,False,False,0,0,0,...,False,True,0.171864,negative,0.8330,positive,True,False,none,4
2,tony530119,2025年04月27日 18:28,NaN,1小時,請問廣大的脆友，有神人可以告知這首音樂的名稱嗎？ 感謝您,False,False,1,0,0,...,False,False,0.509120,neutral,0.8171,positive,False,False,none,4
3,sirblcak,2025年04月27日 20:16,NaN,37分鐘,Dating tip: Just pretend you're interested.,False,False,9,2,0,...,False,False,0.500000,neutral,0.8289,positive,True,False,none,4
4,he_ojung,2025年04月27日 19:11,NaN,23小時,昨天一擊唱完結束主持人來結尾 然後我前面有兩個直接離場 他們一定超後悔吧,False,False,4,0,0,...,False,True,0.246243,negative,0.7558,negative,True,False,mixed,3


In [13]:
import pandas as pd
from transformers import pipeline

# ✅ 載入資料
df = pd.read_csv("test_half_cleaned.csv").dropna(subset=["content"])
df["lang"] = "Ch"

# ✅ 中文 transformer 模型
zh_classifier = pipeline(
    "sentiment-analysis",
    model="uer/roberta-base-finetuned-jd-binary-chinese",
    device=-1
)

# ✅ emoji 字典
emoji_sentiment_dict = {
    '😭': 0.4, '🥹': 0.5, '😂': 0.8, '🤣': 0.9, '🏻': 0.5, '😍': 0.9, '❤': 0.9, '🥲': 0.3,
    '🙏': 0.5, '🔥': 0.9, '🙂': 0.3, '🥺': 0.7, '✨': 0.7, '😅': 0.7, '🏽': 0.5, '🤔': 0.5,
    '💔': 0.9, '😌': 0.5, '😠': 0.1, '♂': 0.5, '😆': 0.9, '🍀': 0.5, '😮': 0.3, '👍': 0.9,
    '👏': 0.9, '😢': 0.2, '👀': 0.5, '💨': 0.5, '😇': 0.8, '♀': 0.5, '🤦': 0.3, '🙄': 0.2,
    '🥰': 0.9, '🤡': 0.5, '💕': 0.9, '🥳': 0.9, '😱': 0.2, '😀': 0.5, '😳': 0.3, '😵': 0.2,
    '🥵': 0.3, '🎉': 0.5, '💗': 0.9, '💚': 0.8, '😔': 0.3, '🤬': 0.1, '📸': 0.5, '😉': 0.5,
    '😊': 0.9, '😃': 0.9, '🏀': 0.5, '🚬': 0.5, '💖': 0.9, '🤭': 0.5, '👌': 0.7, '♥': 0.8,
    '💙': 0.8, '🌸': 0.5, '💀': 0.3, '🏾': 0.5, '👎': 0.2, '🤷': 0.3, '🍁': 0.5, '💫': 0.5,
    '😁': 0.9, '😄': 0.9, '✌': 0.5, '🤩': 0.9, '🙈': 0.8, '😓': 0.3, '🐣': 0.5, '👧': 0.6,
    '💰': 0.8, '🎀': 0.5, '💧': 0.5, '🤪': 0.6, '😶': 0.3, '🤙': 0.7, '💁': 0.5, '✔': 0.5,
    '🧋': 0.5, '😝': 0.7, '🙋': 0.5, '😣': 0.2, '❓': 0.5, '💢': 0.2, '🖕': 0.2, '🙃': 0.3,
    '💪': 0.7, '🏼': 0.5, '🙌': 0.7, '🔗': 0.5, '😤': 0.2, '🚝': 0.5, '💥': 0.9, '😘': 0.9,
    '🤧': 0.3, '☺': 0.5, '👩': 0.5, '💸': 0.2
}

# ✅ 文字模型推論
def classify_transformer_sentiment(text):
    try:
        result = zh_classifier(text)[0]
        label = result['label'].lower()
        score = result['score']
        return label, score
    except:
        return "neutral", 0.5

# ✅ emoji 調整邏輯（只用第一個 emoji）
def adjust_sentiment_by_emoji(text, base_score):
    first_emoji = next((c for c in text if c in emoji_sentiment_dict), None)
    if not first_emoji:
        return base_score

    emoji_value = emoji_sentiment_dict[first_emoji]
    if emoji_value >= 0.8:
        return min(1.0, base_score + 0.4)
    elif emoji_value <= 0.2:
        return max(0.5, base_score - 0.4)
    else:
        return base_score  # 中立 emoji 不動

# ✅ 加總邏輯
def get_sentiment_score_and_label(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.5, "negative"

    label, raw_score = classify_transformer_sentiment(text)

    # 分數映射：正面 => 0.5~1.0，負面 => 0.5~0.0
    if "1" in label or "pos" in label:
        base_score = 0.5 + raw_score * 0.5
    elif "0" in label or "neg" in label:
        base_score = 0.5 - raw_score * 0.5
    else:
        base_score = 0.5

    # 加入 emoji 調整（上下調整最多 0.3）
    final_score = adjust_sentiment_by_emoji(text, base_score)

    # 🚩 使用 0.8 門檻做二元分類
    final_label = "positive" if final_score > 0.8 else "negative"

    return round(final_score, 4), final_label



# ✅ 批次套用
df[["sentiment_score", "sentiment_label"]] = df["content"].apply(
    lambda x: pd.Series(get_sentiment_score_and_label(x))
)

# ✅ 儲存
df.to_csv("test_half_with_sentiment_withemoji.csv", index=False, encoding="utf_8_sig")
print("✅ 輸出完成，檔名：test_half_with_sentiment_withemoji.csv")
df.head()

Device set to use cpu


✅ 輸出完成，檔名：test_half_with_sentiment_withemoji.csv


,author,post_time,topic,time_info,content,has_photo,has_video,like_count,reply_count,repost_count,...,has_exclaim,is_subjective,sent_score_avg,sent_label_avg,sentiment_score,sentiment_label,is_subjective_post,is_satire_humor_simple,person_view,person_view_encoded
0,x730_.166,2025年04月28日 14:56,NaN,3小時,好想聽我不想你想你了的現場喔喔喔,False,False,0,0,0,...,False,True,0.847772,positive,0.8215,positive,True,False,mixed,3
1,vocalenn,2025年04月28日 11:47,KTPOP2025,7小時,kktix 沒有這次高雄拼盤的任何資訊欸 是還沒放上去嗎😦,False,False,5,0,0,...,False,False,0.162927,negative,0.8921,positive,False,False,none,4
2,xin_0121_1023,2025年04月27日 23:06,NaN,19小時,對追星女來說 五月真的大事很妙😋,False,False,1,0,0,...,False,True,0.993290,positive,0.9879,positive,True,False,none,4
3,peiyi7329,2025年04月27日 15:15,NaN,4小時,這屆學測統測會考都有考的考生： 多做多得經歷過擬社會互動又有邊界感的52赫茲鯨魚,False,False,4,1,0,...,False,False,0.984554,positive,0.9736,positive,False,False,none,4
4,yy.c22_,2025年04月28日 09:49,NaN,9小時,昨天夢到廖持修開在林口體育館⋯我等你🥺@chih_siou,False,False,12,1,0,...,False,True,0.796113,positive,0.8364,positive,True,False,mixed,3


In [14]:
# ✅ 二元分類訓練流程 + 上取樣 + 修正版加權損失函數（class weights）

import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils import resample
from collections import Counter
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    TrainingArguments, Trainer
)

# ✅ 載入資料並建立 label（1 = positive, 0 = 非正面）
train_df = pd.read_csv("train_half_with_sentiment_withemoji.csv")
test_df = pd.read_csv("test_half_with_sentiment_withemoji.csv")

label_map = {"positive": 1, "neutral": 0, "negative": 0}
train_df["label"] = train_df["sentiment_label"].map(label_map)
test_df["label"] = test_df["sentiment_label"].map(label_map)
train_df = train_df.dropna(subset=["content", "label"])
test_df = test_df.dropna(subset=["content", "label"])

# ✅ Oversampling 非正面類別（0 類）
df_pos = train_df[train_df["label"] == 1]
df_neg = train_df[train_df["label"] == 0]
df_neg_upsampled = resample(df_neg, replace=True, n_samples=len(df_pos), random_state=42)

train_df_balanced = pd.concat([df_pos, df_neg_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# ✅ 自訂 Dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# ✅ 初始化 tokenizer 與模型（num_labels=2 for 二元分類）
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

train_dataset = SentimentDataset(train_df_balanced["content"].tolist(), train_df_balanced["label"].tolist(), tokenizer)
test_dataset = SentimentDataset(test_df["content"].tolist(), test_df["label"].tolist(), tokenizer)

# ✅ 訓練參數
training_args = TrainingArguments(
    output_dir="./bert_sentiment_ckpt",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    do_eval=True,
    eval_steps=200,
    save_steps=200,
    logging_dir='./logs',
    logging_steps=50,
    num_train_epochs=3,
    save_total_limit=1
)

# ✅ class weights（根據樣本數自動計算）
label_counts = Counter(train_df_balanced["label"])
total = sum(label_counts.values())
class_weights = torch.tensor(
    [total / label_counts[0], total / label_counts[1]],
    dtype=torch.float
)

# ✅ 自訂 Trainer：修正 compute_loss() 的簽名
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# ✅ 評估指標
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro')
    }

# ✅ 開始訓練
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

# ✅ 批次預測 + 儲存
preds = trainer.predict(test_dataset)
pred_labels = preds.predictions.argmax(-1)
test_df["predicted_label"] = pred_labels
test_df.to_csv("test_predictions_with_bert_binary_weighted.csv", index=False, encoding="utf_8_sig")
print("✅ 預測結果已儲存為 test_predictions_with_bert_binary_weighted.csv")

# ✅ 顯示評估報表
print("\n📊 Classification Report")
print(classification_report(test_df["label"], test_df["predicted_label"]))
print("\n🧩 Confusion Matrix")
print(confusion_matrix(test_df["label"], test_df["predicted_label"]))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/chenghaowei/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.671200
100,0.383200


/Users/chenghaowei/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ 預測結果已儲存為 test_predictions_with_bert_binary_weighted.csv

📊 Classification Report
              precision    recall  f1-score   support

           0       0.36      0.26      0.30       103
           1       0.83      0.88      0.85       407

    accuracy                           0.76       510
   macro avg       0.59      0.57      0.58       510
weighted avg       0.73      0.76      0.74       510


🧩 Confusion Matrix
[[ 27  76]
 [ 48 359]]
